In [166]:
import os
from selenium import webdriver
import io
from PIL import Image
import requests
import time
import hashlib
import re
searches_txt = open('searches.txt', 'r')
DRIVER = 'chromedriver'

In [147]:
#make a directory if it doesnt exist for each of the searches in .txt
searches_arr = []
for line in searches:
    print(line)
    search = line.replace('\n', '')
    print(search)
    searches_arr.append(search)
    if not os.path.exists(search):
        os.makedirs(search)
    else:
        print('Directory exists')

In [158]:
wd = webdriver.Chrome(executable_path=DRIVER)

In [113]:
search_box = wd.find_element_by_css_selector('input.gLFyf')
search_box.send_keys('Dogs')

In [167]:
def fetch_image_urls(query:str, max_links:int, wd:webdriver,sleep:int=1):
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(sleep)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links:
        scroll_to_end(wd)

        # get all image thumbnail results
        thumbnail_results = wd.find_elements_by_css_selector("img.rg_i.Q4LuWd.tx8vtf")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail such that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep)
            except Exception:
                continue
            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                if (actual_image.get_attribute('src') and re.match('^http$',actual_image.get_attribute('src')[:4])):
                    print(actual_image.get_attribute('src')[:4] )
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(1)
#             load_more_button = wd.find_element_by_css_selector(".ksb")
#             if load_more_button:
#                 wd.execute_script("document.querySelector('.ksb').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)
    return image_urls

In [168]:
def persist_image(folder_path:str,url:str):
    try:
        image_content = requests.get(url).content

    except Exception as e:
        print(f"ERROR - Could not download {url} - {e}")

    try:
        image_file = io.BytesIO(image_content)
        image = Image.open(image_file).convert('RGB')
        file_path = os.path.join(folder_path,hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
        with open(file_path, 'wb') as f:
            image.save(f, "JPEG", quality=85)
        print(f"SUCCESS - saved {url} - as {file_path}")
    except Exception as e:
        print(f"ERROR - Could not save {url} - {e}")

In [169]:
def search_and_download(search_term:str,driver_path:str,target_path='./images',number_images=5):
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    with webdriver.Chrome(executable_path=driver_path) as wd:
        res = fetch_image_urls(search_term, number_images, wd=wd, sleep=0.5)
        
    for elem in res:
        persist_image(target_folder,elem)

In [171]:
search_term = 'Dog licking -cartoon'
search_and_download(search_term=search_term, driver_path=DRIVER)

Found: 200 search results. Extracting links from 0:200
http
http
http
http
http
Found: 5 image links, done!
SUCCESS - saved https://www.cesarsway.com/wp-content/uploads/2019/05/AdobeStock_153430865.jpeg - as ./images\dog_licking_-cartoon\0f4fe13708.jpg
SUCCESS - saved https://imageserver.petsbest.com/marketing/blog/licking_dog.jpg - as ./images\dog_licking_-cartoon\2c0c6001f7.jpg
SUCCESS - saved https://www.rover.com/blog/wp-content/uploads/2019/04/Licking-dog-960x540.jpg - as ./images\dog_licking_-cartoon\274f0037a5.jpg
SUCCESS - saved https://cdn1-www.dogtime.com/assets/uploads/2016/04/dog-licking.jpg - as ./images\dog_licking_-cartoon\d6e7dc95f5.jpg
SUCCESS - saved https://www.pethealthnetwork.com/sites/default/files/why-does-my-dog-lick-so-much-126832325.jpg - as ./images\dog_licking_-cartoon\2bf48f37c6.jpg
